<a href="https://colab.research.google.com/github/nakajinbee/zero-tuku-deep-learning-2/blob/main/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import numpy as np

# def sigmoid(x):
#     return 1 /(1 + np.exp(-x))

# x = np.random.randn(10, 2)
# w1 = np.random.randn(2, 4)
# b1 = np.random.randn(4)
# w2 = np.random.randn(4, 3)
# b2 = np.random.randn(3)

# h = np.dot(x,w1) + b1
# print("hの値：")
# print(h)
# a = sigmoid(h)
# print("aの値：")
# print(a)
# s = np.dot(a, w2) + b2
# print("sの値")
# print(s)

In [71]:
# Sigmoidレイア 
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None

    def forward(self, x):
        out = 1/(1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 * self.out) * self.out
        return dx

In [104]:
# 全結合層レイアであるAffineレイア
class Affine:
    def __init__(self, w, b):
        self.params = [w, b]
        self.grads  = [np.zeros_like(w), np.zeros_like(b)]
        self.x = None

 
    def forward(self,x):
        w, b = self.params
        out = np.dot(x, w) + b
        self.x = x
        return out

    def backward(self, dout):
        w, b = self.params
        dx = np.dot(dout, w.T)
        dw = np.dot(self.x.T, dout) 
        db = np.sum(dout, axis=0)

        self.grads[0][...] = dw
        self.grads[1][...] = db
        return dx


In [93]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size

        # 重みとバイアスの初期化
        w1 = 0.01 * np.random.randn(I,H)
        b1 = np.zeros(H)
        w2 = 0.01 * np.random.randn(H,O)
        b2 = np.zeros(0)

        self.layers = [
                       Affine(w1, b1),
                       Sigmoid(),
                       Affine(w2, b2)
        ]

        self.loss_layer = SoftmaxWithLoss()


        # 全ての重みをリストにまとめる
        self.params, self.grads = [],[]
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def forward(self, x, t):
        score = self.predict(x)
        loss = self.loss_layer.forward(score, t)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    


In [94]:
# x = np.random.randn(10, 2)
# model = TwoLayerNet(2, 4, 3)
# s = model.predict(x)

# print(s)

In [95]:
class MatMul:
    def __init__(w):
        self.params = [w]
        self.grads = [np.zeros_like(w)]
        self.x = None

    def forward(self, x):
        w, = self.params
        out = np.dot(x,w)
        self.x = x
        return out

    def backward(self, dout):
        w, = self.params
        dx = np.dot(dout, w.T)
        dw = np.dot(self.x.T, dout)
        self.grads[0][...] = dw
        return dx


In [96]:
class Softmax:

    def softmax(x):
        if x.ndim == 2:
            x = x - x.max(axis=1, keepdims=True)
            x = np.exp(x)
            x /= x.sum(axis=1, keepdims=True)
        elif x.ndim == 1:
            x = x - np.max(x)
            x = np.exp(x) / np.sum(np.exp(x))

        return x

    def __init__(self):
        self.params, self.grads = [], []
        self.out = None

    def forward(self, x):
        self.out = softmax(x)
        return self.out

    def backward(self, dout):
        dx = self.out * dout
        sumdx = np.sum(dx, axis=1, keepdims=True)
        dx -= self.out * sumdx
        return dx

誤差逆伝播法によって勾配を求めた
⇨勾配を用いてニューラルネットワークのパラメータを更新する

step1 ミニバッチ：訓練データの中からランダムに複数のデータを選び出す

step2 勾配の算出：誤差逆伝播法により、各重みパラメータに関する損失関数の勾配をもとめる

step3 パラメータの更新：勾配を使って重みパラメータを更新する

step4 繰り返す：１２３を必要な回数だけ繰り返す

In [97]:
# SDG : Stochastic Gradient Decent (確率的勾配降下法)
# lr : learning rate 学習係数

class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
  
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]
  

In [98]:
import numpy as np


def load_data(seed=1984):
    np.random.seed(seed)
    N = 100  # クラスごとのサンプル数
    DIM = 2  # データの要素数
    CLS_NUM = 3  # クラス数

    x = np.zeros((N*CLS_NUM, DIM))
    t = np.zeros((N*CLS_NUM, CLS_NUM), dtype=np.int64)

    for j in range(CLS_NUM):
        for i in range(N):#N*j, N*(j+1)):
            rate = i / N
            radius = 1.0*rate
            theta = j*4.0 + 4.0*rate + np.random.randn()*0.2

            ix = N*j + i
            x[ix] = np.array([radius*np.sin(theta),
                              radius*np.cos(theta)]).flatten()
            t[ix, j] = 1

    return x, t

In [99]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None  # softmaxの出力
        self.t = None  # 教師ラベル

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        # 教師ラベルがone-hotベクトルの場合、正解のインデックスに変換
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)

        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size

        return dx

In [100]:
import matplotlib.pyplot as plt

x, t = load_data()
print('x', x.shape)
print('t', t.shape)

x (300, 2)
t (300, 3)


学習用のソースコード

In [ ]:
# ハイパーパラメータの設定
max_epoch = 300
batch_size = 30
hidden_size = 10
learning_rate = 1.0

x, t = load_data()
model = TwoLayerNet(input_size=2, hidden_size=hidden_size, output_size=3)
optimizer = SGD(lr=learning_rate)

# 学習で使用する変数
data_size = len(x)
max_iters = data_size // batch_size
total_loss = 0
loss_count = 0
loss_list = []

for epoch in range(max_epoch):
    # データのシャッフル
    idx = np.random.permutation(data_size)
    x = x[idx]
    t = t[idx]

    for iters in range(max_iters):
        batch_x = x[iters*batch_size:(iters+1)*batch_size]
        batch_t = t[iters*batch_size:(iters+1)*batch_size]
        
        # 勾配を求め、パラメータを更新
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)

        total_loss += loss
        loss_count += 1

        # 定期的に学習経過を出力
        if (iters+1) % 10 == 0:
            avg_loss = total_loss / loss_count
            print('| epoch %d |  iter %d / %d | loss %.2f'
                  % (epoch + 1, iters + 1, max_iters, avg_loss))
            loss_list.append(avg_loss)
            total_loss, loss_count = 0, 0
            
            
            
# 学習結果のプロット
plt.plot(np.arange(len(loss_list)), loss_list, label='train')
plt.xlabel('iterations (x10)')
plt.ylabel('loss')
plt.show()

# 境界領域のプロット
h = 0.001
x_min, x_max = x[:, 0].min() - .1, x[:, 0].max() + .1
y_min, y_max = x[:, 1].min() - .1, x[:, 1].max() + .1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
X = np.c_[xx.ravel(), yy.ravel()]
score = model.predict(X)
predict_cls = np.argmax(score, axis=1)
Z = predict_cls.reshape(xx.shape)
plt.contourf(xx, yy, Z)
plt.axis('off')

# データ点のプロット
x, t = spiral.load_data()
N = 100
CLS_NUM = 3
markers = ['o', 'x', '^']
for i in range(CLS_NUM):
    plt.scatter(x[i*N:(i+1)*N, 0], x[i*N:(i+1)*N, 1], s=40, marker=markers[i])
plt.show()